Muslimov Arthur, Last Checkpoint: 03/23/2020  </br>
                 Last Checkpoint: 12/16/2021

В главе 2 мы подробно рассмотрели средства доспута к значениям  <br/>
массив NumPy: индексацию (`arr[2, 1]`), срезы (`arr[0, 1:5]`),  <br/>
маскирование (`arr[arr > 0]`), "прихотливую" индексацию         <br/>
(`arr[0, [1, 5]]`), а также их комбинации (`arr[:, [1, 5]]`).   <br/>
Здесь мы изучим аналогичные средства доспута к объектам Series  <br/>
и DataFrame библиотеки Pandas. Они схожи с паттернами NumPy.

Начнём мы с простого Series-объекта, после чего перейдём  <br/>
к более сложным случаям DataFrame-данных.

In [1]:
import pandas as pd

%xmode Minimal
%autosave 0

Exception reporting mode: Minimal


Autosave disabled


## Выборка данных из объекта Series

Объекты Series во многом ведут себя как одномерные массивы NumPy и как    <br/>
стандартные словари Python. Сложностей в понимании возникнуть не должно.

### Объект Series как словарь

Метод доступа к значениям в Series перекочевал от словаря.

In [2]:
data = pd.Series([0.25, 0.5, 0.75],
                 index=['a', 'b', 'c'])
data

a    0.25
b    0.50
c    0.75
dtype: float64

In [3]:
data['b']

0.5

Есть даже методы `keys()` и `items()` как и у словаря.

In [4]:
data.keys()

Index(['a', 'b', 'c'], dtype='object')

In [5]:
list(data.items())  # сам метод выдаёт пока неизвестный нам zip формат

[('a', 0.25), ('b', 0.5), ('c', 0.75)]

Синтаксис модификации и расширения у Series такой же как у словаря.

In [6]:
data['d'] = 1  # добавление элемента
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

Если ты знаешь устройство словаря, то понимашь, что вся магия хеш-таблиц  <br/>
остаётся "под капотом", и ты пользуешься удобным интерфейсом от Python.

### Объект Series как одномерный массив

Как и массивы NumPy, Series предоставляет набор базовых механизмов для выборки  <br/>
элементов, т.е. *срезы*, *маскирование* и _"прихотливую" индексацию_.

In [7]:
# Срез посредством явного индекса
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [8]:
# Срез посредством неявного целочисленного индекса
data[0:3]

a    0.25
b    0.50
c    0.75
dtype: float64

In [9]:
# Маскирование
data[(data > 0.3) & (data < 0.8)]  # получается, что Series объект при сравнении притворяется массивом

b    0.50
c    0.75
dtype: float64

In [11]:
# "Прихотливая" индексация
data[['a', 'd']]

a    0.25
d    1.00
dtype: float64

Затруднение тут может вызвать только срезы. При выполнении  <br/>
среза с помощью явных индексов `data['a':'c']` последнее    <br/>
значение включается в срез, а при `data[0:2]` нет.          <br/>
Вспоминаем картину:

![splits](splits.png)

### Индексаторы: loc, iloc и ix

Если ты используешь явные целочисленные индексы, то может произойти казус:

In [12]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data[1]  # явный индекс

'a'

In [13]:
data[1:3]  # неявные индексы, а ты хотел использовать явные

3    b
5    c
dtype: object

В такой ситуации срезы используют неявные индексы. Чтобы      <br/>
избежать этого, придуманы специальные атрибуты-***индексаторы***,  <br/>
позволяющие явным образом применять индексы.

Атрибут `loc` даёт возможность использовать только явные индексы.

In [14]:
data.loc[1]  # использует явно определённые срезы

'a'

In [15]:
data.loc[1:3]  # даже при срезах

1    a
3    b
dtype: object

Атрибут `iloc` используется для неявных срезов, т.е. в обычном стиле Python.

In [16]:
data.iloc[1]  # только неявные индексы

'b'

In [17]:
data.iloc[1:3]

3    b
5    c
dtype: object

Третий атрибут-индексатор `ix` - это что-то среднее между первых двух.  <br/>
Он эквивалентен обычной индексации с помощью `[]`. Его назначение       <br/>
станет понятнее в контексте объектов DataFrame, о которых далее.

## Выборка данных из объекта DataFrame

При выборке значений из Dataframe стоит иметь ввиду, что он одновременно  <br/>
ведёт себя и как двумерный массив NumPy, и как словарь объектов Series.

### Объект DataFrame как словарь

Как ты уже неоднократно видел, доступ к значениям  <br/> 
у объектов Pandas схож со словарным.

In [27]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127, 'Florida': 19552860,
                        'Illinois': 12882135})
# Воскрешаем нашего подопытного
states = pd.DataFrame({'area': area,
                       'pop': population})
states

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Но, ты ещё не знал того, что имена столбцов автоматически становятся атрибутами.  </br>
****на самом деле об этом уже писалось

In [28]:
states.area  # это уже не похоже на словарь

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Фактически, мы обращаемя к тому же объекту.

In [29]:
states.area is states['area']  # показатель эффективности структуры

True

Однако на это не стоит всегда полагаться. Если в объекте DataFrame  <br/>
уже есть метод или атрибут с таким именем, то обращаться мы будем   <br/>
именно к ним, а не к столбцу. В DataFrame уже есть метод `pop()`.

In [30]:
states.pop is states['pop']  # а они так похожи...

False

Это значит, что не стоит пользоваться атрибутами при присваивании:     <br/>
`states.pop = z` - не делай так! `states['pop'] = z` - так уже лучше.

Добавляются новые столбцы тоже синтаксисом словаря.

In [31]:
states['density'] = states['pop'] / states['area']  # посмотрим на перенаселение
states

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


### Объект DataFrame как двумерный массив

DataFrame - это прокаченный двумерный массив.       <br/>
Взглянем на исходный массив наших данных `states`:

In [32]:
states.values  # ты уже понял что это значит?

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Мы можем выполнить множество привычных действий в облегчённой форме. Например,  <br/>
транспонировать весь DataFrame, т.е. поменять местами строки и столбцы.

In [33]:
states.T  # или states.values.transpose()

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Но, стоит тебе понадобиться взять отдельную строку отсюда, тут же возникают  <br/>
проблемы. Ты не можешь просто взять и написать `states['California']`, т.к.  <br/>
по синтаксису словаря тебе доступны только столбцы. Т.е. нам необходим       <br/>
ещё один тип синтаксиса, аналогчиный индексации массивов. К счастью, нам     <br/>
доступны ранее упомянутые индексаторы `loc`, `iloc` и `ix`. С помощью        <br/>
индексатора `iloc` можно обращаться с объектом DataFrame как с обычным       <br/>
массивом, т.е. используя неявные индексы. Но результатом остаётся DataFrame. 

In [34]:
states.iloc[:3, :2]  # теперь нам доступен и выбор строк

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


In [37]:
states.loc[:'New York', :'pop']  # вертим как хотим! Доступ к строкам здесь тоже есть

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


А здесь индекатор `ix`, в отличии от контекста Series, действительно полезен.

In [38]:
states.ix[:3, :'pop']  # неявный и явный индекс. Вертим с удвоенной силой!

AttributeError: 'DataFrame' object has no attribute 'ix'

*Видим большущее предупреждение, и понимаем, что с `ix` нам скоро придётся прощаться.* </br>
****попрощались

Но помни, что с индекатором `ix` могут всплыть те же проблемы,  <br/>
что и с целочисленными индексами объектов Series.

Эти индекаторы дают нам возможность использовать сленг NumPy.

In [39]:
states.loc[states.density > 100, ['pop', 'density']]  # портал в мир NumPy со всеми его плюшками

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Конечно же можно не только просматривать содержимое, но и изменять его.

In [40]:
states.iloc[0, 2] = 70  # занижаем показатели, чтобы все думали о хорошем
states

,area,pop,density
California,423967,38332521,70.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Чтобы ты чувствовал себя как рыба в воде во всём этом многообразии,  <br/>
я рекомендую тебе поэксперементировать с разными возможностями       <br/>
этих структур данных. Например, использовать разные типы индексов    <br/>
строк и столбцов, а затем пытаться сделать с ними срезы.

*Ну я вот, например, заметил, что список не может стать именем столбца, хотя индеком легко.*

*Ну и ещё - если в стобце появляется `NaN`, то целочисленные элементы становятся типа float. Как оказалось, `np.NaN` имеет тип `float`.*

### Дополнительный синтаксис для индексации

Есть также ещё несколько удобных фишек выборки, не очень вписывающихся в стиль  <br/>
Pandas.   Во первых, *индексация* относится к столбцам, а *срезы* к строкам.    <br/>
****т.к. срезы создаются через .loc[]

In [41]:
states['Texas':'New York']  # да, неочевидно, но, зато, удобно

,area,pop,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


In [43]:
states[1:3]  # фишка работает как с явными, так и с неявными индексами

,area,pop,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Вторая особенность - это то, маскирование тоже  <br/>
работает построчно, а не по столбцам.

In [44]:
states[[True, True, False, False, False]]  # булевая маска накладывается на строки

,area,pop,density
California,423967,38332521,70.00000
Texas,695662,26448193,38.01874
